# Map Elements

In [ ]:
import os

from felt_python import (
    create_map,
    delete_map,
    list_elements,
    list_element_groups,
    get_element_group,
    upsert_elements,
    delete_element,
    upsert_element_groups
)

os.environ["FELT_API_TOKEN"] = "<YOUR_API_TOKEN>"

# Maps

Create a map and retrieve its details, storing the ID in a variable

In [ ]:
resp = create_map(
    title="A felt-python map for testing elements",
    lat=40,
    lon=-3,
    zoom=8,
    public_access="private",
)
map_id = resp["id"]
resp["url"]

# Create elements

Create pins in Madrid and Barcelona

In [ ]:
# GeoJSON with points for Madrid and Barcelona
geojson_feature_collection = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [-3.70379, 40.416775]},
            "properties": {"name": "Madrid"},
        },
        {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [2.173403, 41.385063]},
            "properties": {"name": "Barcelona"},
        },
    ],
}

upsert_elements(map_id, geojson_feature_collection)

# List elements

In [ ]:
elements = list_elements(map_id)
elements

# Update elements

Update the Barcelona pin's color to blue

In [ ]:
# Update the Barcelona element, making it blue
barcelona_element = [el for el in elements["features"] if el["properties"]["name"] == "Barcelona"][0]
barcelona_element_id = barcelona_element["properties"]["felt:id"]

barcelona_element["properties"]["felt:color"] = "#0000FF"
barcelona_feature_collection = {"type": "FeatureCollection", "features": [barcelona_element]}
upsert_elements(map_id, barcelona_feature_collection)

# List Element Groups

First, list any existing element groups (should be empty for a new map)

In [ ]:
existing_groups = list_element_groups(map_id)
existing_groups

# Create element groups

In [ ]:
multiple_groups = [
    {
        "name": "Spanish cities",
        "symbol": "monument",
        "color": "#A02CFA"
    },
    {
        "name": "Parks",
        "symbol": "tree",
        "color": "#00AA55"
    }
]

created_groups = upsert_element_groups(map_id, multiple_groups)
created_groups

In [ ]:
# Get all element groups after creation
all_groups = list_element_groups(map_id)
all_groups

# Add previously-created elements to a group

Assign elements to the "Spanish Cities" group by adding `felt:parentId` property

In [ ]:
# Get the ID of the first group ("Spanish Cities")
element_group_id = all_groups[0]["id"]

# Add all elements to this group
for feature in elements["features"]:
    feature["properties"]["felt:parentId"] = element_group_id

# Update the elements
upsert_elements(map_id, elements)

# List elements in a specific group

In [ ]:
group_elements = get_element_group(map_id, element_group_id)
group_elements

# Create more elements with group assignment

Create elements directly assigned to a group

In [ ]:
# Get the Parks group ID (should be the second group)
parks_group_id = all_groups[1]["id"]

# Create park elements with group assignment
parks_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [-3.6762, 40.4153]},
            "properties": {
                "name": "Retiro Park",
                "felt:parentId": parks_group_id
            },
        },
        {
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [2.1526, 41.3851]},
            "properties": {
                "name": "Parc de la Ciutadella",
                "felt:parentId": parks_group_id
            },
        },
    ],
}

upsert_elements(map_id, parks_geojson)

# Delete the Barcelona element

In [ ]:
delete_element(map_id, barcelona_element_id)

# Verify Barcelona element is gone

In [ ]:
updated_elements = list_elements(map_id)
barcelona_elements = [el for el in updated_elements["features"] if el["properties"].get("name") == "Barcelona"]
print(f"Barcelona elements found: {len(barcelona_elements)}")

## Delete map

In [ ]:
delete_map(map_id)